### Data Prep

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("lib/datasets/twitch_gamers/large_twitch_edges.csv")
df

,numeric_id_1,numeric_id_2
0,98343,141493
1,98343,58736
2,98343,140703
3,98343,151401
4,98343,157118
...,...,...
6797552,97507,29359
6797553,71175,12020
6797554,151702,128281
6797555,118034,38021


## Random walk with Restart.
When the amount of nodes gets to 150 we randomly select a new node and then repeat

In [3]:
import random
def sub_graph(df, query, unique):
    nodes = []
    nodes.append(query)
    # random walk w/ restart
    groups1 = df.groupby("numeric_id_1")
    groups2 = df.groupby("numeric_id_2")
    end = True
    while len(nodes) < 5000:
        while len(nodes) < 150:
            end = True
            curr = query
            while curr in nodes and end:
                selected = list(groups1.get_group(curr)["numeric_id_2"]) + list(groups2.get_group(curr)["numeric_id_1"])
                ind = random.choice(selected)
                if ind not in df["numeric_id_1"].unique():
                    end = False
                elif ind not in df["numeric_id_2"].unique():
                    end = False
                curr = ind
            if ind not in nodes:
                nodes.append(ind)
        query =  np.random.randint(0,123518)
        while query not in unique:
            query =  np.random.randint(0,123518)
        nodes.append(query)

    return nodes

In [4]:
sub = sub_graph(df, 1, df["numeric_id_1"].unique())

In [5]:
selected_df = df[df["numeric_id_1"].isin(sub)]
selected_df = selected_df[selected_df["numeric_id_2"].isin(sub)]
selected_df

,numeric_id_1,numeric_id_2
4496,58736,104152
4510,58736,52703
4549,58736,81907
4560,58736,93147
4565,58736,120471
...,...,...
6782934,69001,9837
6782942,24635,9548
6783235,2731,23000
6783272,61161,58676


In [6]:
key = list(set(list(selected_df["numeric_id_1"]) + list(selected_df["numeric_id_2"])))
len(key)

4243

In [7]:
P = np.zeros((len(key),len(key)))
tmp = selected_df.groupby('numeric_id_1')
for i in range(len(key)):
    if key[i] in list(selected_df["numeric_id_1"]):
        P[i][np.isin(key,tmp.get_group(key[i])['numeric_id_2'])] = 1 / len(tmp.get_group(key[i])['numeric_id_2'])
    else:
        P[i] = np.full(len(key), 1 / len(key))

### PPR

In [8]:
c = .85
n = P.shape[0]
query = 1
q = np.zeros((n, 1))
q[query - 1][0] = 1
r = np.ones((n, 1)) / n
rback = np.zeros((n, 1))
while np.sum(np.absolute(r - rback)) > 10 ** -6:
    rback = r
    r = c * np.dot(P.T, r) + (1 - c) * q

### SALSA

In [9]:
tmp = (-r.reshape((n,))).argsort()[:200]
ha = df[df["numeric_id_1"].isin(np.array(key)[tmp])]

In [10]:
bi = pd.DataFrame()
bi["Hubs"] = ha["numeric_id_1"]
bi["Auth"] = ha["numeric_id_2"]
bi.to_csv("lib/datasets/bi.csv", index=False)

In [62]:
bi = pd.read_csv("lib/datasets/bi.csv")
bi

,Hubs,Auth
0,110905,133370
1,110905,157585
2,110905,4459
3,110905,9315
4,110905,162285
...,...,...
16287,47231,58159
16288,38713,152546
16289,38713,74973
16290,106691,77571


In [11]:
key = np.unique(bi)
L = np.zeros((len(key), len(key)))
tmp = bi.groupby('Hubs')
for i in range(len(key)):
    if key[i] in list(bi["Hubs"]):
        L[i][np.isin(key,tmp.get_group(key[i])['Auth'])] = 1

In [12]:
Lr = np.zeros((len(key), len(key)))
for i in range(len(key)):
    if L[i].sum() != 0:
        Lr[i] = L[i] / L[i].sum()
Lr

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [13]:
Lc = np.zeros((len(key), len(key)))
for i in range(len(key)):
    if L[:,i].sum() != 0:
        Lc[:,i] = L[:,i] / L[:,i].sum()
Lc

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [14]:
rc = Lr.dot(Lc.T)
rc_key = key[~np.all(rc == 0, axis=1)]
H = rc[~np.all(rc == 0, axis=1)]
H = H[:, ~np.all(H == 0, axis=0)]

cr = Lc.T.dot(Lr)
cr_key = key[~np.all(cr == 0, axis=1)]
A = cr[~np.all(cr == 0, axis=1)]
A = A[:, ~np.all(A == 0, axis=0)]
print(H.shape, A.shape)

(200, 200) (11338, 11338)


In [18]:
tmp = np.array([[0, 1, 1, 0],
                [0, 1, 1, 0]])
tmp[0, np.all(tmp == 0, axis=0)] = 1
tmp

array([[1, 1, 1, 1],
       [0, 1, 1, 0]])

### Check connected
check if something appears once on right and then if what it's connected to on the left appears once then disconnected

In [73]:
class DFS:
    # init function to declare class variables
    def __init__(self, V):
        self.V = V
        self.visited = np.zeros(V.shape[0])
        self.vis = np.zeros(V.shape)

    def test(self):
        curr = 1
        for i in range(len(self.V)):
            if self.visited[i] == 0:
                unique = np.unique(self.visited[~np.all(self.V[i] == 0, axis=0)])

                if unique[0] == 0:
                    self.visited[~np.all(self.V[i] == 0, axis=0)] = curr
                    curr += 1
                else:
                    self.visited[~np.all(self.V[i] == 0, axis=0)] = int(unique[1])

    def extract(self):
        self.test()
        cc = []
        print(self.visited)
        for i in range(int(self.visited.max())):
            cc.append(np.where(self.visited == i + 1))
        return cc

In [18]:
class Graph:

    # init function to declare class variables
    def __init__(self, V):
        self.V = V
        self.adj = [[] for i in range(V)]

    def DFSUtil(self, temp, v, visited):

        # Mark the current vertex as visited
        visited[v] = True

        # Store the vertex to list
        temp.append(v)

        # Repeat for all vertices adjacent
        # to this vertex v
        for i in self.adj[v]:
            if visited[i] == False:

                # Update the list
                temp = self.DFSUtil(temp, i, visited)
        return temp

    # method to add an undirected edge
    def addEdge(self, v, w):
        self.adj[v].append(w)
        self.adj[w].append(v)

    # Method to retrieve connected components
    # in an undirected graph
    def connectedComponents(self):
        visited = []
        cc = []
        for i in range(self.V):
            visited.append(False)
        for v in range(self.V):
            if visited[v] == False:
                temp = []
                cc.append(self.DFSUtil(temp, v, visited))
        return cc

In [25]:
g = Graph(len(A))
for i in range(len(A)):
    for j in np.nonzero(A[i])[0]:
        if i != j:
            g.addEdge(i,j)

In [74]:
g = DFS(A)
A_cc = g.extract()

[1. 1. 1. ... 1. 1. 1.]


In [75]:
A_cc

[(array([    0,     1,     2, ..., 14384, 14385, 14386], dtype=int64),)]

In [23]:
from scipy.linalg import eig

S, U = eig(A.T)
stationary = np.array(U[:, np.where(np.abs(S - 1.) < 1e-8)[0][0]].flat)
stationary = stationary / np.sum(stationary)
stationary

array([0.00025132+0.j, 0.00012566+0.j, 0.00012566+0.j, ...,
       0.00012566+0.j, 0.00012566+0.j, 0.00012566+0.j])

In [26]:
(stationary).max()

(0.0005026395234617779+0j)

In [38]:
idx = (-stationary).argsort()[:5]
tmp = cr_u[idx]
tmp

array([104231,  83176, 104384, 159889,   6729], dtype=int64)

In [37]:
cr_u

array([     2,      8,     44, ..., 168079, 168090, 168103], dtype=int64)

In [4]:
P = np.zeros((10000, 10000))
count = 0
for index, values in reduced_df.iterrows():
    P[values["numeric_id_1"] - 1][np.array(values["numeric_id_2"]) - 1] = 1 / len(values["numeric_id_2"])
    print(P[values["numeric_id_1"] - 1].sum())

,Hubs,Auth
0,9008,113885
1,9008,5240
2,9008,47049
3,9008,163802
4,9008,142846
...,...,...
7970,3778,88523
7971,2562,89426
7972,2562,68970
7973,6467,86519
